This analysis script is meant to help in designing to prompt for the autotagger, emphasizing changing lignuistics and trends.

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
import string

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv('/content/polarized_tagged_comments.csv')
df.head()

,CommentContent,Source DB,Class
0,“Its ok to be racist to Jews globally because ...,Israel-Palestine Reddit,Pro-Israel
1,Hamas has broken practically ever ceasefire ev...,Israel-Palestine Reddit,Pro-Israel
2,you're fucking insane. You want evidence of Ha...,Israel-Palestine Reddit,Pro-Israel
3,Israel is the only state that will face accusa...,Israel-Palestine Reddit,Pro-Israel
4,"Gaza isn't all of Palestine, and people in the...",Israel-Palestine Reddit,Pro-Palestine


In [3]:
# Preprocess text
def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    translator = str.maketrans('', '', string.punctuation)
    text = text.lower().translate(translator)
    return ' '.join([word for word in text.split() if word not in stop_words])

df['processed_comment'] = df['CommentContent'].apply(preprocess_text)
df.head()

,CommentContent,Source DB,Class,processed_comment
0,“Its ok to be racist to Jews globally because ...,Israel-Palestine Reddit,Pro-Israel,“its ok racist jews globally war israel” liter...
1,Hamas has broken practically ever ceasefire ev...,Israel-Palestine Reddit,Pro-Israel,hamas broken practically ever ceasefire ever n...
2,you're fucking insane. You want evidence of Ha...,Israel-Palestine Reddit,Pro-Israel,youre fucking insane want evidence hamas killi...
3,Israel is the only state that will face accusa...,Israel-Palestine Reddit,Pro-Israel,israel state face accusations genocide matter ...
4,"Gaza isn't all of Palestine, and people in the...",Israel-Palestine Reddit,Pro-Palestine,gaza isnt palestine people west bank definitel...


In [7]:
# Vectorize text (word frequency)
vectorizer = CountVectorizer(ngram_range=(1, 2))  # Use 1-grams and 2-grams
X = vectorizer.fit_transform(df['processed_comment'])

# Convert to DataFrame
word_counts = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
word_counts['Class'] = df['Class']

# Group by label and calculate term frequencies
label_word_counts = word_counts.groupby('Class').sum()

# Calculate TF-IDF to identify important words/phrases
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), stop_words='english')
X_tfidf = tfidf_vectorizer.fit_transform(df['processed_comment'])

# Create a DataFrame for TF-IDF scores
tfidf_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
tfidf_df['Class'] = df['Class']

# Calculate the top words for each label
def get_top_words_sparse(tfidf_matrix, feature_names, labels, target_label, top_n=100):
    # Filter rows for the target label
    label_indices = [i for i, label in enumerate(labels) if label == target_label]
    label_matrix = tfidf_matrix[label_indices]

    # Calculate mean TF-IDF for each feature
    mean_tfidf = label_matrix.mean(axis=0).A1  # A1 flattens sparse matrix to array
    top_indices = mean_tfidf.argsort()[::-1][:top_n]
    top_words = [(feature_names[i], mean_tfidf[i]) for i in top_indices]
    return top_words

# Get feature names and labels
feature_names = tfidf_vectorizer.get_feature_names_out()
labels = df['Class'].tolist()

# Display top words for each label
for label in set(labels):
    print(f"Top words/phrases for {label}:")
    top_words = get_top_words_sparse(X_tfidf, feature_names, labels, label, top_n=200)
    for word, score in top_words:
        print(f"{word}: {score:.4f}")
    print()

Top words/phrases for Pro-Israel:
fuck: 0.0856
hamas: 0.0690
fuck hamas: 0.0683
israel: 0.0193
people: 0.0182
hamas fuck: 0.0176
idf: 0.0135
hamasisis: 0.0117
like: 0.0110
civilians: 0.0103
war: 0.0100
hostages: 0.0093
gaza: 0.0087
dont: 0.0086
palestinians: 0.0079
terrorist: 0.0078
terrorists: 0.0076
good: 0.0075
support: 0.0075
fucking: 0.0071
innocent: 0.0069
palestinian: 0.0069
say: 0.0067
think: 0.0066
right: 0.0066
children: 0.0064
want: 0.0061
human: 0.0061
killed: 0.0060
israeli: 0.0060
shields: 0.0056
government: 0.0052
way: 0.0051
supports: 0.0049
human shields: 0.0049
kill: 0.0048
say fuck: 0.0046
dead: 0.0046
know: 0.0046
killing: 0.0045
peace: 0.0045
civilian: 0.0045
yeah: 0.0044
need: 0.0043
man: 0.0042
shit: 0.0042
hope: 0.0042
time: 0.0041
jews: 0.0041
use: 0.0040
supporters: 0.0040
care: 0.0039
palestine: 0.0039
israelis: 0.0039
world: 0.0039
stop: 0.0038
life: 0.0038
point: 0.0038
fuck idf: 0.0038
hamas supporters: 0.0037
really: 0.0037
iran: 0.0037
im: 0.0037
terror: